# ADM Explained

__Pega__

__2023-03-15__

This notebook shows exactly how all the values in an ADM model report are calculated. It also shows how the propensity is calculated for a particular customer.

We use one of the shipped datamart exports for the example. This is a model very similar to one used in some of the ADM PowerPoint/Excel deep dive examples. To load your own data, see the vignette on ADM reporting for examples.

For the example we use one particular model: AutoNew36Months over SMS. You can use your own data and select a different model.

To explain the ADM model report, we use one of the IH predictors as an example. Swap for any other predictor when using different data.



In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"

import sys

sys.path.append("../../../")
import pandas as pd
pd.set_option('display.max_colwidth', 0)

format_binning_derived = {'Positives':'{:.0f}', 'Negatives':'{:.0f}', 'Responses %':'{:.2f}', 'Positives %':'{:.2f}', 'Negatives %':'{:.2f}', 'Propensity':'{:.4f}'}
format_lift = {'Positives': '{:.0f}', 'Negatives': '{:.0f}', 'Lift': '{:.4f}'}
format_z_ratio = {'Positives':'{:.0f}', 'Negatives':'{:.0f}', 'Positives %':'{:.2f}', 'Negatives %':'{:.2f}', 'ZRatio':'{:.4f}'}
format_log_odds = {'Positives':'{:.0f}', 'Negatives':'{:.0f}', 'Positives %':'{:.2f}', 'LogOdds %':'{:.4f}', 'ModifiedLogOdds':'{:.4f}'}
format_classifier = {'Positives':'{:.0f}', 'Negatives':'{:.0f}'}

In [2]:
import polars as pl
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from typing import List

from pdstools import datasets, cdh_utils
from pdstools.plots.plots_plotly import ADMVisualisations


pl.Config.set_fmt_str_lengths(100);

In [3]:
dm = datasets.CDHSample(subset=False)

model = dm.combinedData.filter(
    (pl.col("Name") == "AutoNew36Months") & (pl.col("Channel") == "SMS")
)

modelpredictors = (
    dm.combinedData.join(
        model.select(pl.col("ModelID").unique()), on="ModelID", how="inner"
    )
    .filter(pl.col("EntryType") != "Inactive")
    .with_columns(Action=pl.concat_str(["Issue", "Group"], separator="/"),
                  PredictorName=pl.col("PredictorName").cast(pl.Utf8))
    .collect()
)

predictorbinning = modelpredictors.filter(
    pl.col("PredictorName") == "IH.SMS.Outbound.Accepted.pyHistoricalOutcomeCount"
).sort("BinIndex")

## Model Overview

The selected model is shown below. Only the currently active predictors are used for the propensity calculation, so only showing those.



In [4]:
modelpredictors.select(
    pl.col("Action").unique(),
    pl.col("Channel").unique(),
    pl.col("Name").unique(),
    pl.col("PredictorName").unique().sort().implode()
    .list.join(", ").alias("Active Predictors"),
    (pl.col("Performance").unique() * 100).alias("Model Performance (AUC)"),
).to_pandas().T.set_axis(["Values"], axis=1)


,Values
Action,Sales/AutoLoans
Channel,SMS
Name,AutoNew36Months
Active Predictors,"Classifier, Customer.Age, Customer.AnnualIncome, Customer.BusinessSegment, Customer.CLV_VALUE, Customer.CreditScore, Customer.Gender, Customer.MaritalStatus, Customer.NetWealth, Customer.OrganizationLabel, Customer.Prefix, Customer.WinScore, IH.Email.Outbound.Accepted.pxLastGroupID, IH.Email.Outbound.Accepted.pyHistoricalOutcomeCount, IH.Email.Outbound.Rejected.pxLastGroupID, IH.Email.Outbound.Rejected.pyHistoricalOutcomeCount, IH.SMS.Outbound.Accepted.pxLastGroupID, IH.SMS.Outbound.Accepted.pyHistoricalOutcomeCount, IH.SMS.Outbound.Churned.pxLastGroupID, IH.SMS.Outbound.Loyal.pyHistoricalOutcomeCount, IH.SMS.Outbound.Rejected.pxLastGroupID, IH.Web.Inbound.Accepted.pyHistoricalOutcomeCount, IH.Web.Inbound.Churned.pxLastGroupID, IH.Web.Inbound.Churned.pxLastOutcomeTime.DaysSince, IH.Web.Inbound.Rejected.pxLastGroupID"
Model Performance (AUC),60.5845


## Predictor binning for IH.SMS.Outbound.Accepted.pyHistoricalOutcomeCount


The ADM model report will show predictor binning similar to this, with all displayed data coming from fields in the ADM data mart. In subsequent sections we’ll show how all the data is derived from the number of positives and negatives in each of the bins.



In [5]:
predictorbinning.groupby("PredictorName").agg(
    pl.first("ResponseCount").cast(pl.Int16).alias("# Responses"),
    pl.n_unique("BinIndex").alias("# Bins"),
    (pl.first("PerformanceBin") * 100).alias("Predictor Performance(AUC)"),
).rename({"PredictorName": "Predictor Name"}).transpose(include_header=True).rename(
    {"column": "", "column_0": "Value"}
).to_pandas()


,,Value
0,Predictor Name,IH.SMS.Outbound.Accepted.pyHistoricalOutcomeCount
1,# Responses,5602
2,# Bins,5
3,Predictor Performance(AUC),65.4153


In [6]:
BinPositives = pl.col("BinPositives")
BinNegatives = pl.col("BinNegatives")
sumPositives = pl.sum("BinPositives")
sumNegatives = pl.sum("BinNegatives")

predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    ((BinPositives + BinNegatives) / (sumPositives + sumNegatives))
    .round(3)
    .alias("Responses (%)"),
    BinPositives.alias("Positives"),
    (BinPositives / sumPositives).round(3).alias("Positives (%)"),
    BinNegatives.alias("Negatives"),
    (BinNegatives / sumNegatives).round(3).alias("Negatives (%)"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity (%)"),
    pl.col("ZRatio"),
    pl.col("Lift"),
).to_pandas()


,Range/Symbol,Responses (%),Positives,Positives (%),Negatives,Negatives (%),Propensity (%),ZRatio,Lift
0,MISSING,0.001,0.0,0.000,5.0,0.001,0.0000,-2.237073,0.000000
1,<7.08,0.395,7.0,0.194,2208.0,0.397,0.0032,-3.051114,0.491773
2,"[7.08, 12.04>",0.345,11.0,0.306,1919.0,0.345,0.0057,-0.509054,0.886903
3,"[12.04, 18.04>",0.195,12.0,0.333,1082.0,0.194,0.0110,1.764386,1.706886
4,>=18.04,0.064,6.0,0.167,352.0,0.063,0.0168,1.662827,2.608007


## Bin Statistics

### Positive and Negative ratios

Internally, ADM only keeps track of the total counts of positive and negative responses in each bin. Everything else is derived from those numbers. The percentages and totals are trivially derived, and the propensity is just the number of positives divided by the total. The numbers calculated here match the numbers from the datamart table exactly.

In [7]:
binningDerived = predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    BinPositives.alias("Positives"),
    BinNegatives.alias("Negatives"),
    (((BinPositives + BinNegatives) / (sumPositives + sumNegatives)) * 100)
    .round(2)
    .alias("Responses %"),
    ((BinPositives / sumPositives) * 100).round(2).alias("Positives %"),
    ((BinNegatives / sumNegatives) * 100).round(2).alias("Negatives %"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity"),
)
binningDerived.to_pandas(use_pyarrow_extension_array=True).style.format(
    format_binning_derived
).set_properties(
    color="#0000FF", subset=["Responses %", "Positives %", "Negatives %", "Propensity"]
)

,Range/Symbol,Positives,Negatives,Responses %,Positives %,Negatives %,Propensity
0,MISSING,0,5,0.09,0.00,0.09,0.0000
1,<7.08,7,2208,39.54,19.44,39.67,0.0032
2,"[7.08, 12.04>",11,1919,34.45,30.56,34.48,0.0057
3,"[12.04, 18.04>",12,1082,19.53,33.33,19.44,0.0110
4,>=18.04,6,352,6.39,16.67,6.32,0.0168


### Lift

Lift is the ratio of the propensity in a particular bin over the average propensity. So a value of 1 is the average, larger than 1 means higher propensity, smaller means lower propensity:

In [8]:
Positives = pl.col("Positives")
Negatives = pl.col("Negatives")
sumPositives = pl.sum("Positives")
sumNegatives = pl.sum("Negatives")
binningDerived.select(
    "Range/Symbol",
    "Positives",
    "Negatives",
    (
        (Positives / (Positives + Negatives))
        / (sumPositives / (Positives + Negatives).sum())
    ).alias("Lift"),
).to_pandas().style.format(format_lift).set_properties(
    **{"color": "blue"}, subset=["Lift"]
)

,Range/Symbol,Positives,Negatives,Lift
0,MISSING,0,5,0.0000
1,<7.08,7,2208,0.4918
2,"[7.08, 12.04>",11,1919,0.8869
3,"[12.04, 18.04>",12,1082,1.7069
4,>=18.04,6,352,2.6080


### Z-Ratio

The Z-Ratio is also a measure of the how the propensity in a bin differs from the average, but takes into account the size of the bin and thus is statistically more relevant. It represents the number of standard deviations from the average, so centers around 0. The wider the spread, the better the predictor is.
$$\frac{posFraction-negFraction}{\sqrt(\frac{posFraction*(1-posFraction)}{\sum positives}+\frac{negFraction*(1-negFraction)}{\sum negatives})}$$ 

See the calculation here, which is also included in `cdh_utils' zRatio()` <https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.zRatio>`_

In [9]:
def zRatio(
    posCol: pl.Expr = pl.col("BinPositives"), negCol: pl.Expr = pl.col("BinNegatives")
) -> pl.Expr:
    def getFracs(posCol=pl.col("BinPositives"), negCol=pl.col("BinNegatives")):
        return posCol / posCol.sum(), negCol / negCol.sum()

    def zRatioimpl(
        posFractionCol=pl.col("posFraction"),
        negFractionCol=pl.col("negFraction"),
        PositivesCol=pl.sum("BinPositives"),
        NegativesCol=pl.sum("BinNegatives"),
    ):
        return (
            (posFractionCol - negFractionCol)
            / (
                (posFractionCol * (1 - posFractionCol) / PositivesCol)
                + (negFractionCol * (1 - negFractionCol) / NegativesCol)
            ).sqrt()
        ).alias("ZRatio")

    return zRatioimpl(*getFracs(posCol, negCol), posCol.sum(), negCol.sum())


binningDerived.select(
    "Range/Symbol", "Positives", "Negatives", "Positives %", "Negatives %"
).with_columns(zRatio(Positives, Negatives)).to_pandas().style.format(
    format_z_ratio
).set_properties(
    **{"color": "blue"}, subset=["ZRatio"]
)

,Range/Symbol,Positives,Negatives,Positives %,Negatives %,ZRatio
0,MISSING,0,5,0.00,0.09,-2.2371
1,<7.08,7,2208,19.44,39.67,-3.0511
2,"[7.08, 12.04>",11,1919,30.56,34.48,-0.5091
3,"[12.04, 18.04>",12,1082,33.33,19.44,1.7644
4,>=18.04,6,352,16.67,6.32,1.6628


## Predictor AUC


The predictor AUC is the univariate performance of this predictor against the outcome. This too can be derived from the positives and negatives and
there is  a convenient function in pdstools to calculate it directly from the positives and negatives.

Again, this function is implemented in cdh_utils: [`cdh_utils.auc_from_bincounts()`](https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.auc_from_bincounts)

In [10]:
pos=binningDerived.get_column("Positives").to_numpy()
neg=binningDerived.get_column("Negatives").to_numpy()

o = np.argsort((pos / (pos + neg)))

TNR = np.cumsum(neg[o]) / np.sum(neg)
FPR = np.flip(np.cumsum(neg[o]) / np.sum(neg), axis=0)
TPR = np.flip(np.cumsum(pos[o]) / np.sum(pos), axis=0)
Area = (FPR - np.append(FPR[1:], 0)) * (TPR + np.append(TPR[1:], 0)) / 2
auc = 0.5 + np.abs(0.5-np.sum(Area))

fig = px.line(
    x=TPR, y=TNR,
    labels=dict(x='Specificity', y='Sensitivity'),
    title = f"AUC = {auc.round(3)}",
    width=700, height=700,
    range_x=[1,0],
    template='none'
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=1, x1=0, y0=0, y1=1
)
fig.show()

## Predictor score and log odds

The score is calculated from the log odds which are simply the ratio of the probabilities of positives and negatives. For the actual calculation in ADM this is modified slightly to avoid division-by-zero problems and is written differently to avoid numeric instability as shown below.

In [11]:
N = binningDerived.shape[0]
binningDerived.with_columns(
    LogOdds=(pl.col("Positives %") / pl.col("Negatives %")).log(),
    ModifiedLogOdds=(
        ((Positives + 1 / N).log() - (Positives + 1).sum().log())
        - ((Negatives + 1 / N).log() - (Negatives + 1).sum().log())
    ),
).drop("Responses %", "Propensity").to_pandas().style.format(
    format_log_odds
).set_properties(
    **{"color": "blue"}, subset=["LogOdds", "ModifiedLogOdds"]
)

,Range/Symbol,Positives,Negatives,Positives %,Negatives %,LogOdds,ModifiedLogOdds
0,MISSING,0,5,0.00,0.090000,-inf,1.6537
1,<7.08,7,2208,19.44,39.669998,-0.713262,-0.8141
2,"[7.08, 12.04>",11,1919,30.56,34.480000,-0.120688,-0.2320
3,"[12.04, 18.04>",12,1082,33.33,19.440001,0.539125,0.4264
4,>=18.04,6,352,16.67,6.320000,0.969891,0.8721


## Propensity mapping

### Log odds contribution for all the predictors

To get to a propensity, the log odds of the relevant bins of the active predictors are added up and divided by the number of active predictors +1, then used to index in the classifier.

Below an example. From all the active predictors of the model for we pick a value (in the middle for numerics, first symbol for symbolics) and show the (modified) log odds. These log odds values are averaged (added up and divided by number of active predictors + 1), and this is the “score” that is mapped to a propensity value by the classifier (which is constructed using the PAV(A) algorithm).

In [12]:
def middleBin():
    return pl.col("BinIndex") == (pl.max("BinIndex") / 2).floor().cast(pl.UInt32)


def RowWiseLogOdds(Bin, Positives, Negatives):
    Bin, N = Bin.list.get(0) - 1, Positives.list.lengths()
    Pos, Neg = Positives.list.get(Bin), Negatives.list.get(Bin)
    PosSum, NegSum = Positives.list.sum(), Negatives.list.sum()
    return (
        (((Pos + (1 / N)).log() - (PosSum + N).log()))
        - (((Neg + (1 / N)).log()) - (NegSum + N).log())
    ).alias("Modified Log odds")


df = (
    modelpredictors.filter(pl.col("PredictorName") != "Classifier")
    .groupby("PredictorName")
    .agg(
        Value=pl.when(pl.col("Type").first() == "numeric")
        .then(
            ((pl.col("BinLowerBound") + pl.col("BinUpperBound")) / 2).where(middleBin())
        )
        .otherwise(pl.col("BinSymbol").str.split(",").list.first().where(middleBin())),
        Bin=pl.col("BinIndex").where(middleBin()),
        Positives=pl.col("BinPositives"),
        Negatives=pl.col("BinNegatives"),
    )
    .with_columns(
        pl.col(["Positives", "Negatives"]).list.get(pl.col("Bin").list.get(0) - 1),
        pl.col("Bin", "Value").list.get(0),
        LogOdds=RowWiseLogOdds(pl.col("Bin"), pl.col("Positives"), pl.col("Negatives")),
    )
    .sort("PredictorName")
)
propensity_mapping = df.vstack(
    pl.DataFrame(dict(zip(
                df.columns,
                ["Average log odds"] + [None] * 4 + [df["LogOdds"].sum() / (len(df)+1)],
                )),
    schema=df.schema,
    )
).to_pandas().style.set_properties(**{'color': 'blue'}, subset=["LogOdds"])

propensity_mapping

,PredictorName,Value,Bin,Positives,Negatives,LogOdds
0,Customer.Age,37.16,2.000000,7.000000,3042.000000,-1.103080
1,Customer.AnnualIncome,2827.71,1.000000,7.000000,3401.000000,-1.178083
2,Customer.BusinessSegment,middleSegmentPlus,1.000000,10.000000,3369.000000,-0.825686
3,Customer.CLV_VALUE,352.52,1.000000,7.000000,1379.000000,-0.275516
4,Customer.CreditScore,432.2,1.000000,14.000000,426.000000,1.568775
5,Customer.Gender,M,1.000000,7.000000,1735.000000,-0.505115
6,Customer.MaritalStatus,Single,1.000000,6.000000,1454.000000,-0.475067
7,Customer.NetWealth,5458.64,1.000000,7.000000,2679.000000,-0.939484
8,Customer.OrganizationLabel,NON-MISSING,1.000000,31.000000,3646.000000,0.235670
9,Customer.Prefix,Miss,2.000000,8.000000,1174.000000,-0.021901


## Classifier

The success rate is defined as $\frac{positives}{positives+negatives}$ per bin. 

The adjusted propensity that is returned is a small modification (Laplace smoothing) to this and calculated as $\frac{0.5+positives}{1+positives+negatives}$ so empty models return a propensity of 0.5.


In [13]:
classifier = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        Propensity=(BinPositives / (BinPositives / BinNegatives)),
        AdjustedPropensity=((0.5 + BinPositives) / (1 + BinPositives + BinNegatives)),
    )
    .select(
        [
            pl.col("BinIndex").alias("Index"),
            pl.col("BinSymbol").alias("Bin"),
            BinPositives.alias("Positives"),
            BinNegatives.alias("Negatives"),
            ((pl.cumsum("BinResponseCount") / pl.sum("BinResponseCount")) * 100).alias(
                "Cum. Total (%)"
            ),
            (pl.col("BinPropensity") * 100).alias("Propensity (%)"),
            (pl.col("AdjustedPropensity") * 100).alias("Adjusted Propensity (%)"),
            ((pl.cumsum("BinPositives") / pl.sum("BinPositives")) * 100).alias(
                "Cum Positives (%)"
            ),
            pl.col("ZRatio"),
            (pl.col("Lift") * 100).alias("Lift(%)"),
            pl.col("BinResponseCount").alias("Responses"),
        ]
    )
)
classifier.drop("Responses").to_pandas().style.format(format_classifier).set_properties(
    **{"color": "blue"}, subset=["Adjusted Propensity (%)"]
)

,Index,Bin,Positives,Negatives,Cum. Total (%),Propensity (%),Adjusted Propensity (%),Cum Positives (%),ZRatio,Lift(%)
0,1,<-1.95,0,35,0.624777,0.000000,1.388889,0.000000,-5.934769,0.000000
1,2,"[-1.95, -0.63>",7,1939,35.362373,0.359712,0.385208,19.444445,-2.322612,55.975200
2,3,"[-0.63, -0.5>",7,1331,59.246696,0.523169,0.560120,38.888889,-0.674919,81.410900
3,4,"[-0.5, -0.36>",8,1125,79.471619,0.706090,0.749559,61.111111,0.289246,109.875500
4,5,"[-0.36, -0.29>",4,446,87.504463,0.888889,0.997783,72.222221,0.590078,138.321000
5,6,"[-0.29, -0.22>",3,319,93.252411,0.931677,1.083591,80.555557,0.563599,144.979300
6,7,"[-0.22, 0.99>",2,144,95.858620,1.369863,1.700680,86.111107,0.776338,213.165900
7,8,>=0.99,5,227,100.000000,2.155172,2.360515,100.000000,1.700289,335.368800


## Final Propensity

Below the classifier mapping. On the x-axis the binned scores (log odds values), on the y-axis the Propensity. Note the returned propensities are following a slightly adjusted formula, see the table above. The bin that contains the calculated score is highlighted.

The score -0.11403 falls in bin 7 of the classifier, so for this set of inputs, the model returns a propensity of 1.70%.

In [14]:
score = propensity_mapping.data.loc[24, "LogOdds"]
score_bin = modelpredictors.filter(pl.col("EntryType") == "Classifier").select(
    pl.col("BinSymbol").where(
        (pl.col("BinLowerBound") < score) & (pl.col("BinUpperBound") > score)
    )
)["BinSymbol"][0]


adjusted_propensity = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        AdjustedPropensity=((0.5 + BinPositives) / (1 + BinPositives + BinNegatives)),
    )
    .select(
        pl.col("AdjustedPropensity").where(
            (pl.col("BinLowerBound") < score) & (pl.col("BinUpperBound") > score)
        )
    )["AdjustedPropensity"][0]
    * 100
)
adjusted_propensity = round(adjusted_propensity, 2)


fig = ADMVisualisations.distribution_graph(
    modelpredictors.filter(pl.col("EntryType") == "Classifier"),
    "Propensity distribution",
).add_annotation(
    x=score_bin,
    y=1400,
    text="Returned propensity: 1.7%",
    bgcolor="#FFFFFF",
    bordercolor="#000000",
    showarrow=False,
)
bin_index = list(fig.data[0]["x"]).index(score_bin)
fig.data[0]["marker_color"] = ["grey"] * bin_index + ["#1f77b4"] + ["grey"]
fig